In [ ]:
import pandas as pd


In [ ]:
from datasets import load_dataset
df_raw = load_dataset('Ezi/medical_and_legislators_synthetic')
df_pandas = df_raw['train'].to_pandas()
df_pandas.columns

In [ ]:
df_raw = pd.read_csv('/Users/nullzero/Documents/repos/github.com/privacy-identity/vda-simulation-medical/vda-sim-medical/data/raw/marketing_campaign.csv', sep='\t')
df_raw.shape
df_raw["Response"].unique()
df_raw.rename(columns={'Reponse': 'target'}, inplace=True)

In [ ]:
df = df_raw.copy()

In [ ]:
print(df.describe) 

In [122]:
print(f"shape is {df.shape} and there are {df.isna().sum()} NAs")
print(df.columns)
#columns = ["Unnamed: 0", "suffix", "nickname, "]

shape is (2240, 29) and there are ID                      0
Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Z_CostContact           0
Z_Revenue               0
Response                0
dtype: int64 NAs
Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', '

In [124]:
df.to_csv("./customer_profile_marketing.csv")

In [ ]:
df.head(10)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import seaborn as sns


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

# Drop irrelevant columns
df_bak = df.copy()
df = df_bak

# Handle missing values
df['middle_name'].fillna('None', inplace=True)
df['suffix'].fillna('None', inplace=True)

# Feature engineering - calculate age
df['birthday'] = pd.to_datetime(df['birthday'])
#df['age'] = (pd.Timestamp.now() - df['birthday']).astype('<m8[ns]')

# Categorical encoding
df = pd.get_dummies(df, columns=['gender', 'party', 'state', 'type'])

# Drop columns with all missing values (like washington_post_id.1)
df.dropna(axis=1, how='all', inplace=True)

# Chart age before normalization
sns.boxplot(x=df['age'])
plt.title('Boxplot of Age to Detect Outliers')
plt.show()

# Remove outliers for the age column (optional, if outliers are detected)
q_low = df['age'].quantile(0.1)
q_high = df['age'].quantile(0.90)
#df_filtered = df[(df['age'] > q_low) & (df['age'] < q_high)]

# Visualization - Distribution of Age with limited bins
sns.histplot(df['age'])  # Reduced bins for better performance
plt.title('Distribution of Age')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()


# Normalize continuous variables


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from datetime import datetime

# Drop irrelevant columns and duplicates
critical_columns = ['last_name', 'first_name', 'age', 'gender', 'state', 'district', \
                    'party', 'twitter', 'facebook', 'youtube']

# Handle missing values
df['middle_name'].fillna('None', inplace=True)
df['suffix'].fillna('None', inplace=True)
print("Suffix and middle name filled with 'None'")

# Feature engineering - calculate age

# Ensure 'birthday' is in datetime format
df['birthday'] = pd.to_datetime(df['birthday']).dt.floor('D')

# Calculate the current date
today = pd.Timestamp.now().normalize()

# Calculate age in years
df['age'] = today.year - df['birthday'].dt.year
print(df['age'][0:3])
# Select only critical columns
df_reduced = df.copy()
df_reduced = df[critical_columns]
print(df_reduced.columns)


In [ ]:

# Categorical encoding with LabelEncoder for binary categories
label_encoder = LabelEncoder()
df_reduced['gender'] = label_encoder.fit_transform(df_reduced['gender'])

# Categorical encoding with OneHotEncoder for multi-class categories
df_reduced = pd.get_dummies(df_reduced, columns=['state', 'party', 'district'])

# Handle online presence as binary features (e.g., presence/absence of account)
df_reduced['twitter'] = df_reduced['twitter'].notna().astype(int)
df_reduced['facebook'] = df_reduced['facebook'].notna().astype(int)
df_reduced['youtube'] = df_reduced['youtube'].notna().astype(int)

# Normalize continuous variables
scaler = StandardScaler()
df_reduced[['age']] = scaler.fit_transform(df_reduced[['age']])

# Visualization of the reduced dataframe
print("Reduced DataFrame:")
print(df_reduced.head())

# Correlation matrix to understand the relationship between features
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:

# Missing data visualization
plt.figure(figsize=(12,8))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.title('Missing Data Heatmap')
plt.show()

# Categorical distribution of gender
sns.countplot(x='gender_M', data=df)
plt.title('Distribution of Gender')
plt.xlabel('Gender (1 = Male, 0 = Female)')
plt.ylabel('Count')
plt.show()

# Distribution by Party
sns.countplot(x='party', data=df)
plt.title('Distribution by Party')
plt.xlabel('Party')
plt.ylabel('Count')
plt.show()

plt.figure(figsize=(12,8))
sns.heatmap(df.corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()
